In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set(font_scale = 1.5)

import matplotlib.pyplot as plt
import scikitplot as skplt
from matplotlib.colors import ListedColormap
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('fivethirtyeight')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, make_scorer

In [ ]:
customer_data = pd.read_csv('../data-sources/customer-churn/customer-churn.csv')
customer_data.head()

In [ ]:
customer_data.rename(columns={'customerID': 'CustomerID', 'gender': 'Gender', 'tenure': 'Tenure'}, inplace=True)

In [ ]:
def ColumnTransformer(cell):
    if cell == 0:
        return 'No'
    else:
        return 'Yes'

customer_data['SeniorCitizen'] = customer_data['SeniorCitizen'].apply(ColumnTransformer)

In [ ]:
def ColumnTransformer(cell):
    if cell == 'Electronic check':
        return 'ElCh'
    elif cell == 'Mailed check':
        return 'MaCh'
    elif cell == 'Bank transfer (automatic)':
        return 'BaTr-A'
    else:
        return 'CrCa-A'
    
customer_data['PaymentMethod'] = customer_data['PaymentMethod'].apply(ColumnTransformer)

In [ ]:
customer_data.drop(customer_data[customer_data['Tenure'] == 0].index, inplace=True)
customer_data.drop('CustomerID', axis=1, inplace=True)

customer_data['TotalCharges'] = customer_data['TotalCharges'].astype(float)

In [ ]:
y = customer_data.pop('Churn')

In [ ]:
X = customer_data

In [ ]:
X_dum = pd.get_dummies(X, columns=X.select_dtypes(include='object').columns, drop_first=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.3, random_state=1, stratify=y)

In [ ]:
scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
cust_prec_scorer = make_scorer(precision_score, pos_label='Yes')

In [ ]:
pipe_dt = make_pipeline(RandomOverSampler(random_state=1),
                        DecisionTreeClassifier(random_state=1))

pipe_dt_params = {'decisiontreeclassifier__criterion': ['gini', 'entropy'],
                  'decisiontreeclassifier__max_depth': [5, 10, None],
                  'decisiontreeclassifier__min_samples_split': [2, 5, 9],
                  'decisiontreeclassifier__max_features': [0.3, 0.6]}

pipe_dt_gs = GridSearchCV(pipe_dt, pipe_dt_params, cv=5, scoring=cust_prec_scorer, n_jobs=-3, verbose=1)
pipe_dt_gs.fit(X_train, y_train)

print('Best parameters:', pipe_dt_gs.best_params_)
print('Mean cross-validated score:', pipe_dt_gs.best_score_)
print('Test score:', pipe_dt_gs.score(X_test, y_test))

In [ ]:
dt_predictions = pipe_dt_gs.predict(X_test)
skplt.metrics.plot_confusion_matrix(y_test, dt_predictions, figsize=(8, 8))
plt.show()

In [ ]:
pipe_lr = make_pipeline(RandomOverSampler(random_state=1),
                        LogisticRegression(max_iter=10000))

pipe_lr_params = {'logisticregression__penalty': ['elasticnet'],
                  'logisticregression__C': [0.01, 0.05, 0.5, 1, 10, 100],
                  'logisticregression__l1_ratio': [0.001, 0.01, 0.1, 0.5, 0.9, 0.99],
                  'logisticregression__solver': ['saga'],
                  'logisticregression__fit_intercept': [True, False]}

pipe_lr_gs = GridSearchCV(pipe_lr, pipe_lr_params, cv=5, scoring=cust_prec_scorer, n_jobs=-3, verbose=1)
pipe_lr_gs.fit(X_train, y_train)

print('Best parameters:', pipe_lr_gs.best_params_)
print('Mean cross-validated score:', pipe_lr_gs.best_score_)
print('Test score:', pipe_lr_gs.score(X_test, y_test))

In [ ]:
lr_predictions = pipe_lr_gs.predict(X_test)
skplt.metrics.plot_confusion_matrix(y_test, lr_predictions, figsize=(8, 8))
plt.show()

In [ ]:
pipe_rf = make_pipeline(RandomUnderSampler(random_state=1),
                        RandomForestClassifier(random_state=1))

pipe_rf_params = {'randomforestclassifier__criterion': ['gini', 'entropy'],
                  'randomforestclassifier__max_depth': [5, 10, None],
                  'randomforestclassifier__max_features': [0.3, 0.6, 'auto'],
                  'randomforestclassifier__max_samples': [0.5, 0.8, None],
                  'randomforestclassifier__n_estimators': [30, 80, 130, 180, 230]} 

pipe_rf_gs = GridSearchCV(pipe_rf, pipe_rf_params, cv=5, scoring=cust_prec_scorer, n_jobs=-3, verbose=1)
pipe_rf_gs.fit(X_train, y_train)

print('Best parameters:', pipe_rf_gs.best_params_)
print('Mean cross-validated score:', pipe_rf_gs.best_score_)
print('Test score:', pipe_rf_gs.score(X_test, y_test))

In [ ]:
rf_predictions = pipe_rf_gs.predict(X_test)
skplt.metrics.plot_confusion_matrix(y_test, rf_predictions, figsize=(8, 8))
plt.show()

In [ ]:
pipe_svc = make_pipeline(RandomUnderSampler(random_state=1),
                         SVC(random_state=1))

pipe_svc_params = {'svc__gamma': np.logspace(-5, 2, 6),
                   'svc__C': np.logspace(-2, 3, 6),
                   'svc__kernel': ['rbf', 'sigmoid', 'linear', 'poly'],
                   'svc__break_ties': [True, False]} 

pipe_svc_gs = GridSearchCV(pipe_svc, pipe_svc_params, cv=5, scoring=cust_prec_scorer, n_jobs=-3, verbose=1)
pipe_svc_gs.fit(X_train, y_train)

print('Best parameters:', pipe_svc_gs.best_params_)
print('Mean cross-validated score:', pipe_svc_gs.best_score_)
print('Test score:', pipe_svc_gs.score(X_test, y_test))

In [ ]:
svc_predictions = pipe_svc_gs.predict(X_test)
skplt.metrics.plot_confusion_matrix(y_test, svc_predictions, figsize=(8, 8))
plt.show()